# Random Forest

## Prior Progress
Random Forest 14 got 70% accuracy on 5 classes (we didn't have class 0 yet).

## Next Task
Do the 6-way classification using Nucleus plus RBC.

In [1]:
import datetime
print(datetime.datetime.now())
from platform import python_version
print('Python',python_version())
import numpy as np
import pandas as pd
import sklearn   # pip install --upgrade scikit-learn
print('sklearn',sklearn.__version__)
import tensorflow as tf
tf.config.list_physical_devices('GPU')

2022-06-08 18:31:42.614255
Python 3.8.10
sklearn 1.0.2


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [2]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.utils import shuffle
from sklearn.feature_selection import RFE
import joblib # used to dump/load sklearn models
from CellProfiler_Util import CP_Util
from RandomForestUtil import RF_Util

In [3]:
BASE_DIR='/home/jrm/ShepherdML/TumorII/'
NUCLEUS_FILES=[
    'Nucleus_Rollup_0.csv',
    'Nucleus_Rollup_1.csv',
    'Nucleus_Rollup_2.csv',
    'Nucleus_Rollup_3.csv',
    'Nucleus_Rollup_4.csv',
    'Nucleus_Rollup_5.csv']
RBC_FILES=[
    'RBC_Rollup_0.csv',
    'RBC_Rollup_1.csv',
    'RBC_Rollup_2.csv',
    'RBC_Rollup_3.csv',
    'RBC_Rollup_4.csv',
    'RBC_Rollup_5.csv']
CLASSES=range(0,6)   # for the first time, use all 6 classes
MODELS_DIR='/home/jrm/Adjeroh/Naved/models/RandomForest.16/'

In [4]:
print(datetime.datetime.now())
PREFIX='abc_'
def set_prefix(prefix):
    global PREFIX
    PREFIX=prefix
def get_prefix(col_name):
    global PREFIX
    return PREFIX+col_name  # hard coded for now
def load_all_classes():
    X = None
    y = None
    for i in CLASSES:
        print(datetime.datetime.now())
        print("Loading class",i)
        Xnuc = pd.read_csv(BASE_DIR+NUCLEUS_FILES[i])
        Xnuc.set_index(['PatchNumber'],inplace=True)
        set_prefix('Nuc_')
        Xnuc.rename(get_prefix,axis='columns',inplace=True)
        Xrbc = pd.read_csv(BASE_DIR+RBC_FILES[i])
        Xrbc.set_index(['PatchNumber'],inplace=True)
        set_prefix('Rbc_')
        Xrbc.rename(get_prefix,axis='columns',inplace=True)
        Xi = Xnuc.join(Xrbc,how='outer')
        #Xi.drop(columns=['PatchNumber'],inplace=True)
        size = len(Xi)
        yi = np.ones(size) * i   # e.g. class 3
        if X is None:
            X = Xi
            y = yi
        else:
            X = pd.concat( (X,Xi) )
            y = np.concatenate( (y,yi) )
    X.fillna(0,inplace=True)  
    return X,y
Xtrain,ytrain=load_all_classes()
Xtrain

2022-06-08 18:31:44.596129
2022-06-08 18:31:44.596378
Loading class 0
2022-06-08 18:32:08.736731
Loading class 1
2022-06-08 18:32:17.285657
Loading class 2
2022-06-08 18:32:27.616343
Loading class 3
2022-06-08 18:32:34.856946
Loading class 4
2022-06-08 18:32:39.972945
Loading class 5


,Nuc_ObjectNumber_count,Nuc_ObjectNumber_mean,Nuc_ObjectNumber_std,Nuc_ObjectNumber_min,Nuc_ObjectNumber_25%,Nuc_ObjectNumber_50%,Nuc_ObjectNumber_75%,Nuc_ObjectNumber_max,Nuc_AreaShape_Area_count,Nuc_AreaShape_Area_mean,...,Rbc_Number_Object_Number_75%,Rbc_Number_Object_Number_max,Rbc_Parent_ShrinkRBC_count,Rbc_Parent_ShrinkRBC_mean,Rbc_Parent_ShrinkRBC_std,Rbc_Parent_ShrinkRBC_min,Rbc_Parent_ShrinkRBC_25%,Rbc_Parent_ShrinkRBC_50%,Rbc_Parent_ShrinkRBC_75%,Rbc_Parent_ShrinkRBC_max
PatchNumber,,,,,,,,,,,,,,,,,,,,,
397,7.0,4.0,2.160247,1.0,2.50,4.0,5.50,7.0,7.0,216.000000,...,0.00,0.0,0.0,0.0,0.000000,0.0,0.00,0.0,0.00,0.0
398,1.0,1.0,0.000000,1.0,1.00,1.0,1.00,1.0,1.0,200.000000,...,0.00,0.0,0.0,0.0,0.000000,0.0,0.00,0.0,0.00,0.0
399,1.0,1.0,0.000000,1.0,1.00,1.0,1.00,1.0,1.0,181.000000,...,0.00,0.0,0.0,0.0,0.000000,0.0,0.00,0.0,0.00,0.0
400,4.0,2.5,1.290994,1.0,1.75,2.5,3.25,4.0,4.0,239.250000,...,0.00,0.0,0.0,0.0,0.000000,0.0,0.00,0.0,0.00,0.0
401,4.0,2.5,1.290994,1.0,1.75,2.5,3.25,4.0,4.0,237.500000,...,0.00,0.0,0.0,0.0,0.000000,0.0,0.00,0.0,0.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1587,7.0,4.0,2.160247,1.0,2.50,4.0,5.50,7.0,7.0,368.285714,...,0.00,0.0,0.0,0.0,0.000000,0.0,0.00,0.0,0.00,0.0
1588,17.0,9.0,5.049752,1.0,5.00,9.0,13.00,17.0,17.0,519.352941,...,0.00,0.0,0.0,0.0,0.000000,0.0,0.00,0.0,0.00,0.0
1589,19.0,10.0,5.627314,1.0,5.50,10.0,14.50,19.0,19.0,388.315789,...,1.75,2.0,2.0,1.5,0.707107,1.0,1.25,1.5,1.75,2.0


In [5]:
print(datetime.datetime.now())
print('Cross-validation...')
rf2 = RF_Util()
rf2.set_train(Xtrain,ytrain)
cv_scores = rf2.cross_validation()
print(cv_scores)
print('mean %.4f +/- %.4f' % (cv_scores.mean(),cv_scores.std()))

2022-06-08 18:33:03.128213
Cross-validation...
[0.61804034 0.7281106  0.69938048 0.68910547 0.46426413]
mean 0.6398 +/- 0.0949


In [ ]:
print(datetime.datetime.now())
joblib.dump(rf3.get_mode(),MODELS_DIR)